In [1]:
from typing import Any, Optional
from pathlib import Path
import json
import glob

import numpy as np
import pandas as pd
import spacy
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.enable_notebook()

## Tools

In [4]:
def get_data_path(file: Optional[str] = None) -> Path:
    return Path.joinpath(Path.cwd().parent, "data", file)


def read_data(file: str) -> None:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def export_data(file: str, data: Any) -> None:
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        

def pipe(raw_input: Any, *functions, **functions_with_args) -> Any:
    output = raw_input

    if functions:
        for function in functions:
            output = function(output)

    if functions_with_args:
        for function, args_list in functions_with_args.items():
            output = eval(function)(output, *args_list)

    return output

## Preprocessing Tools

In [5]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
stop_words = stopwords.words("english")


def lemmatize(texts: list[str]) -> list[str]:
    def _lemmatize(text: str, allowed_postags: Optional[list[str]] = None) -> str:
        if allowed_postags is None:
            allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]

        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    return [_lemmatize(text) for text in texts]


def gensim_preprocess(lemmatized_texts: list[str]) -> list[list[str]]:
    def _gensim_preprocess(text: str) -> list[str]:
        return simple_preprocess(text, deacc=True)
    
    return [_gensim_preprocess(text) for text in lemmatized_texts]


def remove_stopwords(texts: list[list[str]]) -> list[list[str]]:
    def _remove_stopwords(words: list[str]) -> list[str]:
        return [word for word in words if word not in stop_words]
    
    return [_remove_stopwords(words) for words in texts]
    

def generate_ngrams(words: list[list[str]]):
    bigram_phrases = gensim.models.Phrases(words, min_count=5, threshold=50)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[words], threshold=50)
    
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)
    
    def _make_bigrams(words: list[list[str]]):
        return (bigram[doc] for doc in words)

    def _make_trigrams(words: list[list[str]]):
        return (trigram[bigram[doc]] for doc in words)
    
    data_bigrams = _make_bigrams(words)
    data_bigrams_trigrams = _make_trigrams(data_bigrams)
    
    return list(data_bigrams_trigrams)


def make_corpus(words: list[list[str]]) -> tuple[list[tuple[int, int]], corpora.dictionary.Dictionary]:
    id2word = corpora.Dictionary(words)
    return [id2word.doc2bow(text) for text in words], id2word


def apply_tfidf(corpus: list[tuple[int, int]], id2word: corpora.dictionary.Dictionary) -> list[tuple[int, int]]:
    tfidf = TfidfModel(corpus, id2word=id2word)
    
    low_value = 0.03
    words = []
    words_missing_in_tfidf = []
    
    for i in range(0, len(corpus)):
        bow = corpus[i]
        low_value_words = []
        
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops = low_value_words + words_missing_in_tfidf

        for item in drops:
            words.append(id2word[item])

        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow
        
    return corpus, id2word


def preprocess_text(texts: list[str]) -> Any:
    return pipe(
        texts,
        lemmatize,
        gensim_preprocess,
        remove_stopwords,
        generate_ngrams,
        make_corpus,
    )

## Data

In [6]:
data = read_data(get_data_path("ushmm_dn.json"))["texts"]

In [9]:
sample = data[:2]

In [13]:
data = pd.read_csv(get_data_path("keychron_K2_reviews.csv"))["content"].values.tolist()[:-1]

### Sample

In [ ]:
sample = data[0]

In [ ]:
half_length = int(len(sample) / 2)

In [ ]:
sample = [sample[:half_length]] + [sample[half_length:]]

## Preprocessing

In [10]:
corpus, id2word = preprocess_text(sample)

In [14]:
corpus, id2word = preprocess_text(data)

### TF-IDF

In [15]:
corpus, id2word = apply_tfidf(corpus, id2word)

## Model

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)

## Visualization

In [19]:
vis = gensimvis.prepare(
    lda_model, 
    corpus, 
    id2word, 
    mds="mmds",
    R=10
)

/Users/d.e.magno/TheCuriousCirc/lda-example/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/d.e.magno/TheCuriousCirc/lda-example/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/d.e.magno/TheCuriousCirc/lda-example/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/d.e.magno/TheCuriousCirc/lda-example/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation 

In [20]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.326519 -0.068743       1        1  78.630880
8     -0.114576 -0.299883       2        1  14.033382
0     -0.091786  0.177770       3        1   2.878188
7      0.172746 -0.100292       4        1   1.345479
9      0.096441  0.000426       5        1   1.078193
5      0.154481  0.109853       6        1   0.752001
3      0.041079  0.119806       7        1   0.396898
1      0.031193  0.008905       8        1   0.357838
4      0.010115  0.026861       9        1   0.297759
2      0.026826  0.025296      10        1   0.229381, topic_info=               Term        Freq       Total Category  logprob  loglift
221            work  228.000000  228.000000  Default  10.0000  10.0000
146         product   56.000000   56.000000  Default   9.0000   9.0000
189          switch  258.000000  258.000000  Default   8.0000   8.0000
612        function   41.000000   41.000000  Default   7.0000   7.0000
646       bluetooth   82.000000   82.000000  Default   6.0000   6.0000
...             ...         ...         ...      ...      ...      ...
1577      variation    0.196647    0.720437  Topic10  -5.4741   4.7791
1357           lock    0.184909    0.708558  Topic10  -5.5357   4.7342
2486        compren    0.161319    0.684971  Topic10  -5.6721   4.6315
2487  continuamente    0.161319    0.684971  Topic10  -5.6721   4.6315
2488     desconecta    0.161319    0.684971  Topic10  -5.6721   4.6315

[139 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1995      5  0.728343   advertise
1697      3  0.987294       apple
2738      6  0.685237     bandaid
7         1  0.629418     battery
7         2  0.203038     battery
...     ...       ...         ...
217       1  0.998498        well
1993      8  0.901385  whatsoever
931       2  0.984828       wired
221       1  0.534656        work
221       2  0.460155        work

[124 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 1, 8, 10, 6, 4, 2, 5, 3])

## Export figure as html

In [ ]:
pyLDAvis.save_html(vis, str(get_data_path("lda_keychron_reviews.html")))